In [1]:
import numpy as np
import pandas as pd
from pandas import Series,DataFrame

## 12.1 分类数据

引子

In [3]:
values = pd.Series(['apple', 'orange', 'apple',
   ....:                     'apple'] * 2)

In [4]:
values.unique()

values.value_counts()

array(['apple', 'orange'], dtype=object)

##### 许多数据系统（数据仓库、统计计算或其它应用）都发展出了特定的表征重复值的方法，以进行高效的存储和计算。在数据仓库中，最好的方法是使用所谓的包含不同值得维表(Dimension Table)，将主要的参数存储为引用维表整数键：

In [6]:
values = pd.Series([0, 1, 0, 0] * 2)
dim = Series(['apple','orange'])

In [7]:
#这种用整数表示的方法称为分类或字典编码表示法
dim.take(values)

0     apple
1    orange
0     apple
0     apple
0     apple
1    orange
0     apple
0     apple
dtype: object

### pandas的分类类型

pandas有一个特殊的分类类型，用于保存使用整数分类表示法的数据。看一个之前的Series例子：

In [8]:
In [20]: fruits = ['apple', 'orange', 'apple', 'apple'] * 2

In [21]: N = len(fruits)

In [22]: df = pd.DataFrame({'fruit': fruits,
   ....:                    'basket_id': np.arange(N),
   ....:                    'count': np.random.randint(3, 15, size=N),
   ....:                    'weight': np.random.uniform(0, 4, size=N)},
   ....:                   columns=['basket_id', 'fruit', 'count', 'weight'])

In [23]: df



,basket_id,fruit,count,weight
0,0,apple,4,3.858126
1,1,orange,13,3.892483
2,2,apple,4,1.944527
3,3,apple,7,0.253229
4,4,apple,11,1.150064
5,5,orange,9,1.692060
6,6,apple,3,1.411141
7,7,apple,12,2.915576


In [9]:
fruit_cat = df['fruit'].astype('category')

In [11]:
fruit_cat

0     apple
1    orange
2     apple
3     apple
4     apple
5    orange
6     apple
7     apple
Name: fruit, dtype: category
Categories (2, object): [apple, orange]

还可以从其它Python序列直接创建pandas.Categorical：

In [12]:
my_categories = pd.Categorical(['foo', 'bar', 'baz', 'foo', 'bar'])

In [13]:
# 如果你已经从其它源获得了分类编码，你还可以使用from_codes构造器
In [34]: categories = ['foo', 'bar', 'baz']

In [35]: codes = [0, 1, 2, 0, 0, 1]

In [36]: my_cats_2 = pd.Categorical.from_codes(codes, categories)



### 用分类进行计算

In [14]:
In [41]: np.random.seed(12345)

In [42]: draws = np.random.randn(1000)

In [43]: draws[:5]

array([-0.20470766,  0.47894334, -0.51943872, -0.5557303 ,  1.96578057])

In [16]:
bins = pd.qcut(draws,4,labels=['q1','q2','q3','q4'])

In [17]:
bins.categories

Index(['q1', 'q2', 'q3', 'q4'], dtype='object')

In [18]:
bins.codes

array([1, 2, 1, 1, 3, 3, 2, 2, 3, 3, 3, 0, 2, 2, 3, 3, 0, 1, 3, 1, 1, 2, 3,
       0, 1, 2, 2, 2, 2, 3, 0, 0, 0, 0, 0, 2, 0, 2, 0, 2, 0, 1, 0, 0, 0, 2,
       2, 0, 2, 3, 2, 2, 1, 3, 3, 0, 0, 2, 3, 1, 3, 2, 2, 3, 3, 0, 1, 0, 1,
       0, 0, 3, 3, 3, 3, 1, 1, 0, 0, 2, 2, 0, 3, 2, 3, 3, 0, 3, 1, 3, 2, 3,
       1, 3, 2, 3, 2, 0, 2, 2, 0, 1, 1, 0, 1, 1, 3, 3, 1, 3, 1, 2, 3, 0, 0,
       1, 2, 1, 3, 0, 0, 0, 2, 3, 1, 1, 2, 1, 1, 1, 0, 0, 3, 3, 2, 2, 3, 1,
       1, 2, 0, 3, 2, 1, 2, 1, 1, 1, 3, 3, 0, 3, 1, 0, 2, 0, 1, 3, 3, 1, 1,
       1, 0, 3, 0, 3, 1, 3, 3, 3, 3, 1, 3, 0, 0, 1, 0, 0, 2, 1, 1, 1, 0, 3,
       3, 3, 0, 1, 1, 1, 0, 0, 3, 2, 3, 3, 0, 0, 3, 2, 3, 1, 2, 1, 0, 0, 0,
       1, 0, 1, 3, 2, 1, 3, 1, 3, 0, 2, 3, 1, 1, 2, 0, 0, 2, 3, 3, 2, 1, 2,
       0, 0, 0, 3, 1, 0, 0, 2, 2, 2, 0, 1, 3, 1, 3, 0, 2, 1, 3, 1, 3, 0, 1,
       0, 2, 0, 1, 1, 0, 2, 0, 3, 1, 3, 0, 0, 1, 2, 1, 2, 2, 1, 0, 2, 2, 1,
       0, 0, 1, 1, 1, 1, 2, 0, 3, 0, 0, 3, 0, 2, 1, 2, 0, 2, 2, 1, 3, 3, 1,
       3, 3,

In [19]:
In [49]: bins = pd.Series(bins, name='quartile')

In [50]: results = (pd.Series(draws)
   ....:            .groupby(bins)
   ....:            .agg(['count', 'min', 'max'])
   ....:            .reset_index())


In [21]:
results

,quartile,count,min,max
0,q1,250,-2.949343,-0.685484
1,q2,250,-0.683066,-0.010115
2,q3,250,-0.010032,0.628894
3,q4,250,0.634238,3.927528


### 12.2 GroupBy高级应用

** 在第10章，我们在分组操作中学习了apply方法，进行转换。还有另一个transform方法，它与apply很像，但是对使用的函数有一定限制：**

- 它可以产生向分组形状广播标量值
- 它可以产生一个和输入组形状相同的对象
- 它不能修改输入

In [22]:
In [75]: df = pd.DataFrame({'key': ['a', 'b', 'c'] * 4,
   ....:                    'value': np.arange(12.)})


In [23]:
 g = df.groupby('key').value

In [25]:
g.mean()

key
a    4.5
b    5.5
c    6.5
Name: value, dtype: float64

In [26]:
 g.transform(lambda x: x.mean())

0     4.5
1     5.5
2     6.5
3     4.5
4     5.5
5     6.5
6     4.5
7     5.5
8     6.5
9     4.5
10    5.5
11    6.5
Name: value, dtype: float64

与apply类似，transform的函数会返回Series，但是结果必须与输入大小相同。举个例子，我们可以用lambda函数将每个分组乘以2：

In [27]:
g.transform(lambda x:x * 2)

0      0.0
1      2.0
2      4.0
3      6.0
4      8.0
5     10.0
6     12.0
7     14.0
8     16.0
9     18.0
10    20.0
11    22.0
Name: value, dtype: float64

### 分组的时间重采样

## 12.3 链式编程技术